# Decision Trees for Machine Learning

## Objective

Learn building blocks of decision trees, including entropy and information gain.

Using Decision Trees with Scikit-Learn

## Data Set

The data set contains individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. 

The target column is whether individuals make less than or equal to 50k a year, or more than 50k a year.

The data set can be downloaded from [the University of California, Irvine's website.](http://archive.ics.uci.edu/ml/datasets/Adult), the column description can be found [here](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names)

## Reading In the Data

In [1]:
import pandas as pd
import numpy as np

income_file = "C:/Users/i7/csv/income.csv"

# Column names, not included in file.
names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']

income = pd.read_csv(income_file, names=names)
income.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
income.shape

(32561, 15)

In [3]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Cleaning the Data

### Converting Categorical Values to Numeric Variables

Column such as 'workclass' are categorical that have string values.

Another example of a column of categories is sex, where the options are Male and Female.

In [4]:
income['workclass'].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [5]:
# Convert a single column from text categories to numbers
col = pd.Categorical(income["workclass"])
income["workclass"] = col.codes
print(income["workclass"].head(5))

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8


In [6]:
# Convert all categorical column to numbers
for name in ["education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"]:
    col = pd.Categorical(income[name])
    income[name] = col.codes

## Creating Splits

Split income into two parts based on the value of the workclass column

    •	private_incomes should contain all rows where workclass is 4.
    •	public_incomes should contain all rows where workclass is not 4.

(the value Private in the workclass column mapped to the numeric code 4)

In [7]:
private_incomes = income[income["workclass"] == 4]
public_incomes = income[income["workclass"] != 4]

print(private_incomes.shape)
print(public_incomes.shape)

(22696, 15)
(9865, 15)


## Overview Of Data Set Entropy

Entropy refers to disorder. The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the 'income' column would have low entropy.

In [8]:
# Compute the entropy of the 'income' column in the income dataframe
import math
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

prob_0 = income[income["income"] == 0].shape[0] / income.shape[0]
prob_1 = income[income["income"] == 1].shape[0] / income.shape[0]
income_entropy = -(prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1, 2))
print(income_entropy)

0.9709505944546686
0.7963839552022132


## Information Gain

information gain tells which split will reduce entropy the most.

In [9]:
# Compute the information gain for splitting on the age column of income.
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that the function matches the answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

income_entropy = calc_entropy(income["income"])

median_age = income["age"].median()

left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

age_information_gain = income_entropy - ((left_split.shape[0] / income.shape[0]) * calc_entropy(left_split["income"]) + ((right_split.shape[0] / income.shape[0]) * calc_entropy(right_split["income"])))

0.970950594455
0.170950594455


## Finding The Best Split

calculating which split would have the highest information gain.

In [10]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that the answer is the same as on the last screen
# print(calc_information_gain(income, "age", "income"))

columns = ["age", "workclass", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "hours-per-week", "native-country"]

information_gains = []
# Loop through and compute information gains
for col in columns:
    information_gain = calc_information_gain(income, col, "income")
    information_gains.append(information_gain)

# Find the name of the column with the highest gain
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
print(highest_gain)

marital-status


## Using Decision Trees With Scikit-Learn

In [11]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
columns = ["age", "workclass", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "hours-per-week", "native-country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)


clf.fit(income[columns], income["income"])


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

## Splitting The Data Into Train And Test Sets

Make 80% of rows training data, and the rest testing data.

In [12]:
# Set a random seed so the shuffle is the same every time
np.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(np.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

## Evaluating Error With AUC

AUC ranges from 0 to 1, so it's ideal for binary classification. The higher the AUC, the more accurate the predictions.

In [13]:
from sklearn.metrics import roc_auc_score

predictions = clf.predict(test[columns])
error = roc_auc_score(test["income"], predictions)
print(error)

0.942643158141


## Computing Error On The Training Set

The AUC for the predictions on the testing set is about .942. Let's compare this against the AUC for predictions on the training set to see if the model is overfitting.

In [14]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train["income"], predictions))

0.942550296569


## Exploring Decision Tree Variance

In [15]:
np.random.seed(1)

# Generate a column containing random numbers from 0 to 4
income["noise"] = np.random.randint(4, size=income.shape[0])

# Adjust "columns" to include the noise column
columns = ["noise", "age", "workclass", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "hours-per-week", "native-country"]

# Make new train and test sets
train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["income"], train_predictions)

print(test_auc)
print(train_auc)


0.691406001394
0.975076161435


##### The random noise column causes significant overfitting. The test set accuracy decreases to .691, and the training set accuracy increases to .975.

## Pruning Leaves To Prevent Overfitting

Pruning involves building a full tree, and then removing the leaves that don't add to prediction accuracy